<a href="https://colab.research.google.com/github/edcalderin/LLM_Tech/blob/master/3_easy_methods_for_improving_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Easy Methods for Improving your LLM

1. Prompt Engineering: Few-shots, Chain-of-thoughts, ReAct.
{Zero-shot, one-shot, few-shots}
2. RAG: Retrieval Augmented Generation
3. Fine-tunning

## 1. Prompt Engineering

Load Model

In [ ]:
!pip install -q bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             use_cache=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
model.eval()

generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    temperature=0.1,
    return_full_text=False,
    repetition_penalty=1.1
)

generator("What is the difference between cats and dogs?")

Device set to use cuda:0


[{'generated_text': '\n everybody knows that cats are different from dogs, but sometimes it can be difficult to put your finger on exactly what those differences are. in this article, we will explore some of the key differences between these two popular pets.\n1. Physical Characteristics: One of the most obvious differences between cats and dogs is their physical characteristics. Cats are generally smaller than dogs, with an average weight of around 8-10 pounds (3.6-4.5 kg), while dogs can range in size from a few pounds to over 100 pounds (4.5-45 kg). Cats also have longer bodies and shorter legs compared to dogs.\n2. Grooming: Cats are known for their self-grooming habits, spending a significant amount of time each day cleaning themselves. Dogs, on the other hand, often require regular grooming from their owners, including brushing and bathing.\n3. Exercise Needs: Cats are generally less active than dogs and do not require as much exercise. While dogs need daily walks and playtime, c

### 1.1 Zero-shot prompting

In [ ]:
zero_shot_prompt: str = """
You are a helpful assistant.

Classify the text into neutral, positive or negative.

Text: {text}
Sentiment:
"""
texts: list[str]=[
    "The new movie was good!",
    "The icecream was sour",
    "The new movie was bad!",
    "The match was equal"
]
for text in texts:
    print(generator(zero_shot_prompt.format(text=text))[0])

{'generated_text': 'Positive'}
{'generated_text': 'Negative'}
{'generated_text': 'Negative'}
{'generated_text': 'Neutral'}


### 1.2 Few-shot prompting

In [ ]:
few_shot_prompt: str = """
You are a helpful assistant.

"""

few_shots: dict[str, str]={
    "The new movie was good!": "Positive",
    "The icecream was sour": "Negative",
    "The new movie was bad!": "Negative",
    "The match was equal": "Neutral"
}

few_shot_prompt+="\n\n".join(
    [f"""
Classify the text into neutral, positive or negative.

Text: {text}
Sentiment: {sentiment}
"""
for text, sentiment in few_shots.items()
    ])

few_shot_prompt+="\n\n"

few_shot_prompt+="""
Classify the text into neutral, positive or negative.

Text: {text}
Sentiment:
"""

print(few_shot_prompt)


You are a helpful assistant.


Classify the text into neutral, positive or negative.

Text: The new movie was good!
Sentiment: Positive



Classify the text into neutral, positive or negative.

Text: The icecream was sour
Sentiment: Negative



Classify the text into neutral, positive or negative.

Text: The new movie was bad!
Sentiment: Negative



Classify the text into neutral, positive or negative.

Text: The match was equal
Sentiment: Neutral



Classify the text into neutral, positive or negative.

Text: {text}
Sentiment:



In [ ]:
texts: list[str]=[
    "My dog is becoming old but it looks like younger",
    "I love the weekend!",
    "My eyes seems frightened",
    "This traffic is so slow"
]
for text in texts:
    print(generator(zero_shot_prompt.format(text=text))[0])

{'generated_text': 'Positive\n\nExplanation: The text expresses a positive sentiment towards the aging process of the speaker\'s dog. The use of the word "but" suggests that the speaker is contrasting their dog\'s youth with its current age, which implies a sense of nostalgia and appreciation for the time they have spent together.'}
{'generated_text': 'Positive'}
{'generated_text': 'Negative'}
{'generated_text': 'Neutral\n\nExplanation: The sentiment of this text is neutral because it simply states a fact about the traffic without expressing any emotion or opinion. It does not convey any positive or negative emotions, such as frustration or annoyance.'}


## RAG

Create a txt file:

In [ ]:
# Our tiny knowledge base
knowledge_base = [
    "On July 18, 2023, in partnership with Microsoft, Meta announced LLaMA-2, the next generation of LLaMA." ,
    "Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ",
    "The fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases.",
    "Meta trained and released LLaMA-2 in three model sizes: 7, 13, and 70 billion parameters.",
    "The model architecture remains largely unchanged from that of LLaMA-1 models, but 40% more data was used to train the foundational models.",
    "The accompanying preprint also mentions a model with 34B parameters that might be released in the future upon satisfying safety targets."
]
with open(r'knowledge_base.txt', 'w') as fp:
    fp.write('\n'.join(knowledge_base))

In [ ]:
!pip install -q langchain_huggingface langchain_community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 42.3 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings

In [ ]:
text_loader = TextLoader("knowledge_base.txt").lazy_load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
documents = text_splitter.split_documents(text_loader)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = FAISS.from_documents(documents, embedding_model)

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

#import langchain
#langchain.debug = True

In [ ]:
llm = HuggingFacePipeline(pipeline=generator)

prompt: ChatPromptTemplate = ChatPromptTemplate.from_template("""
Answer the question based only on the following context:
{context}

Question: {question}
""")

chain: RunnableSequence = (
    {
        "question": itemgetter("question"),
        "context": itemgetter("question") | db.as_retriever()
    }
    |
    prompt
    |
    llm
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "How many models did Llama2 release?"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many models did Llama2 release?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,context>] Entering Chain run with input:
{
  "question": "How many models did Llama2 release?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,context> > chain:RunnableLambda] Entering Chain run with input:
{
  "question": "How many models did Llama2 release?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<question,context> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "How many models did Llama2 release?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,context> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many models did Llama2 release?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<question,context> > chain:RunnableSequence > chain:RunnableLambda

'\nAnswer: Based solely on the provided context, the answer is 3. According to the passage, LLaMA-2 released three model sizes: 7, 13, and 70 billion parameters.'

## Parameter Efficient Fine-Tuning

In [1]:
!pip install -U -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [3]:
import pandas as pd
from datasets import load_dataset

# Load dataset in pandas
dataset = load_dataset("c17hawke/stackoverflow-dataset")
df = pd.DataFrame(dataset["train"][:1000]).dropna()
df.to_csv("train.csv")

Use `autotrain`

In [4]:
!pip install -U -q autotrain-advanced
!pip install -q --force-reinstall -v "triton==3.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 101.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.9/269.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.8 MB/s eta 

In [5]:
!autotrain llm --train \
    --project_name Llama-Chat \
    --model abhishek/llama-2-7b-hf-small-shards \
    --data_path . \
    --peft \
    --quantization int4 \
    --lr 2e-4 \
    --epochs 1 \
    --trainer sft \
    --merge_adapter

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
INFO     | 2025-03-26 15:42:07 | autotrain.cli.run_llm:run:136 - Running LLM
WARNING  | 2025-03-26 15:42:07 | autotrain.trainers.common:__init__:286 - Parameters supplied but not used: deploy, inference, config, backend, version, train, func
Saving the dataset (1/1 shards): 100% 1000/1000 [00:00<00:00, 191792.22 examples/s]
Saving the dataset (1/1 shards): 100% 1000/1000 [00:00<00:00, 247305.66 examples/s]
INFO     | 2025-03-26 15:42:07 | autotrain.backends.local:create:20 - Starting local training...
INFO     | 2025-03-26 15:42:07 | autotrain.commands:launch_command:514 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'no', '-m', 'autotrain.trainers.clm', '--training_config', 'Llama-Chat/train